Name: Shirui Chen
ID: 6958613091

In [1]:
import sklearn
from sklearn.datasets import *
from sklearn import tree
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import random
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

In [2]:
def encode_record_into_vector(record):
    categorical_cols = df.select_dtypes(include='object')
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(categorical_cols).toarray())
    encoder.get_feature_names()
    encoder_df.columns = encoder.get_feature_names()
    final_df = df.join(encoder_df) 
    final_df.drop(categorical_cols, axis=1, inplace=True)
    x = pd.DataFrame(final_df.iloc[record,:])
    return np.array(x)

In [3]:
def parse_file_into_matrix(file_name):
    global df
    global x
    global y
    df = pd.read_csv(file_name, delimiter = ";")
    y = pd.DataFrame(df['y'].map({'yes': 1, 'no': 0}))
    y = np.asmatrix(y)
    df.drop('y',axis=1, inplace=True)
    record = df.index.tolist()
    x = np.asmatrix(encode_record_into_vector(record))
    return (x, y)

In [4]:
parse_file_into_matrix("/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW2/bank-additional-full.csv")

(matrix([[ 56., 261.,   1., ...,   0.,   1.,   0.],
         [ 57., 149.,   1., ...,   0.,   1.,   0.],
         [ 37., 226.,   1., ...,   0.,   1.,   0.],
         ...,
         [ 56., 189.,   2., ...,   0.,   1.,   0.],
         [ 44., 442.,   1., ...,   0.,   1.,   0.],
         [ 74., 239.,   3., ...,   1.,   0.,   0.]]),
 matrix([[0],
         [0],
         [0],
         ...,
         [0],
         [1],
         [0]]))

In [5]:
print(x.shape)
print(y.shape)

(41188, 63)
(41188, 1)


D = 41188
P = 63

In [6]:
sum(y == 1)

matrix([[4640]])

In [7]:
sum(y == 0)

matrix([[36548]])

Q1
There are 4640 positive instances and 36548 negative instances in the dataset.

def x_y_for_running_tests():
    x, y = load_digits(2, True) 
    return x, y

x, y=x_y_for_running_tests()
print(x)
print(y)

In [8]:
def training_testing_split(x, y, training_ratio):
    y_df = pd.DataFrame(y)
    y_df.columns = ['y']
    y1 = y_df[y_df['y'] == 1]
    y0 = y_df[y_df['y'] == 0]
    s1 = np.array(y1.index)
    s0 = np.array(y0.index) 
    random.shuffle(s1)
    random.shuffle(s0)
    train1 = s1[:int(len(s1)*training_ratio)]
    test1 = s1[int(len(s1)*training_ratio):]
    train0 = s0[:int(len(s0)*training_ratio)]
    test0 = s0[int(len(s0)*training_ratio):]
    x_train = np.concatenate([x[train1], x[train0]])
    y_train = np.concatenate([y[train1], y[train0]])
    x_test = np.concatenate([x[test1], x[test0]])
    y_test = np.concatenate([y[test1], y[test0]])
    return (x_train, y_train, x_test, y_test)

In [9]:
x_train, y_train, x_test, y_test = training_testing_split(x, y, 0.8)
print(sum(y_train == 1)/len(y_train))
print(sum(y_test == 1)/len(y_test))

[[0.11265554]]
[[0.1126487]]


In [10]:
x_train, y_train, x_test, y_test = training_testing_split(x, y, 0.5)
print(sum(y_train == 1)/len(y_train))
print(sum(y_test == 1)/len(y_test))

[[0.11265417]]
[[0.11265417]]


In [11]:
x_train, y_train, x_test, y_test = training_testing_split(x, y, 0.3)
print(sum(y_train == 1)/len(y_train))
print(sum(y_test == 1)/len(y_test))

[[0.11265782]]
[[0.11265261]]


In [12]:
x_train, y_train, x_test, y_test = training_testing_split(x, y, 0.1)
print(sum(y_train == 1)/len(y_train))
print(sum(y_test == 1)/len(y_test))

[[0.11267606]]
[[0.11265174]]


Q2
The ratio of positive instances in the test dataset to the total number of instances in the test dataset is 0.11.

In [13]:
def train_models(x_train,y_train,model):
    if model == 'decision_tree':
        clf = tree.DecisionTreeClassifier()
        clf = clf.fit(x_train, y_train)
        return clf
    elif model == 'naive_bayes':
        clf = BernoulliNB()
        clf = clf.fit(x_train, y_train)
        return clf
    elif model == 'linear_classifier':
        clf = make_pipeline(StandardScaler(), SGDClassifier(loss='squared_loss'))
        clf.fit(x_train, y_train)
        return clf
    elif model == 'support_vector_classification':
        clf = svm.SVC()
        clf = clf.fit(x_train, y_train)
        return clf

In [14]:
def evaluate_model(x_test, y_test, model):
    y_predict = model.predict(x_test)
    return sklearn.metrics.f1_score(y_test, y_predict)

In [26]:
def table_gen(ratio):
    dt=[]
    nb=[]
    lc=[]
    for i in range(1,11,1):
        np.random.seed(i)
        x_train, y_train, x_test, y_test=training_testing_split(x, y, ratio)
        model1=train_models(x_train,y_train,'decision_tree')
        f1_dt=evaluate_model(x_test, y_test, model1)
        dt.append(f1_dt)
        model2=train_models(x_train,y_train,'naive_bayes')
        f1_nb=evaluate_model(x_test, y_test, model2)
        nb.append(f1_nb)
        model3=train_models(x_train,y_train,'linear_classifier')
        f1_lc=evaluate_model(x_test, y_test, model3)
        lc.append(f1_lc)
    table=pd.concat([pd.DataFrame(dt),pd.DataFrame(nb),pd.DataFrame(lc)],axis=1)
    table.columns=['decision_tree','naive_bayes','linear_classifier']
    table.index.names=['trial']
    table.index += 1
    return table

In [27]:
df1 = table_gen(0.1)
df1

,decision_tree,naive_bayes,linear_classifier
trial,,,
1,0.503719,0.373518,0.220461
2,0.490603,0.384022,0.229786
3,0.495598,0.366310,0.206076
4,0.489517,0.368221,0.163798
5,0.504231,0.375768,0.206934
6,0.515899,0.378195,0.212745
7,0.504597,0.371211,0.191650
8,0.504675,0.369038,0.184846
9,0.486187,0.380212,0.203440


In [28]:
df2 = table_gen(0.3)
df2

,decision_tree,naive_bayes,linear_classifier
trial,,,
1,0.524343,0.381388,0.160423
2,0.515092,0.374466,0.137006
3,0.530485,0.372336,0.198196
4,0.494839,0.375176,0.171574
5,0.509272,0.375409,0.206275
6,0.514320,0.374851,0.219141
7,0.518031,0.379347,0.187084
8,0.509938,0.373731,0.182348
9,0.504942,0.378337,0.146944


In [29]:
df3 = table_gen(0.5)
df3

,decision_tree,naive_bayes,linear_classifier
trial,,,
1,0.525299,0.395020,0.155352
2,0.508991,0.369497,0.207066
3,0.514650,0.378788,0.195657
4,0.508846,0.367795,0.166355
5,0.509638,0.378998,0.213154
6,0.504551,0.385480,0.186960
7,0.521091,0.370820,0.464652
8,0.511349,0.387808,0.144465
9,0.519805,0.381163,0.208780


In [30]:
df4 = table_gen(0.7)
df4

,decision_tree,naive_bayes,linear_classifier
trial,,,
1,0.528809,0.409497,0.197537
2,0.502356,0.366450,0.213747
3,0.511514,0.394178,0.184003
4,0.513820,0.394427,0.145087
5,0.533523,0.381070,0.223032
6,0.522946,0.376865,0.136488
7,0.523066,0.373878,0.195846
8,0.505407,0.382362,0.196743
9,0.508778,0.366853,0.181402


In [31]:
df5 = table_gen(0.9)
df5

,decision_tree,naive_bayes,linear_classifier
trial,,,
1,0.540481,0.375114,0.205660
2,0.506864,0.384758,0.194628
3,0.528616,0.382544,0.182270
4,0.521186,0.388246,0.170624
5,0.522466,0.400371,0.210486
6,0.470718,0.372401,0.164309
7,0.492179,0.362292,0.125609
8,0.566879,0.375912,0.200447
9,0.472222,0.366300,0.216006


H0: Naive bayes is same as decision tree using the f-measure metric.

Ha: Naive bayes is better than decision tree using the f-measure metric.

The test I used is paired test, since we are comparing the results of three different models but using the same sample each time.

In [32]:
t1,p1=stats.ttest_rel(df1['decision_tree'], df1['naive_bayes'], alternative = 'less')
t1,p1

(35.381901117547095, 0.99999999997155)

When ratio is 0.1, p = 0.99 > 0.05, we cannot reject H0 and cannot accept Ha.

In [33]:
t2,p2=stats.ttest_rel(df2['decision_tree'], df2['naive_bayes'], alternative = 'less')
t2,p2

(39.623441638130885, 0.9999999999896697)

When ratio is 0.3, p = 0.99 > 0.05, we cannot reject H0 and cannot accept Ha.

In [34]:
t3,p3=stats.ttest_rel(df3['decision_tree'], df3['naive_bayes'], alternative = 'less')
t3,p3

(44.30986569820096, 0.999999999996205)

When ratio is 0.5, p = 0.99 > 0.05, we cannot reject H0 and cannot accept Ha.

In [35]:
t4,p4=stats.ttest_rel(df4['decision_tree'], df4['naive_bayes'], alternative = 'less')
t4,p4

(31.50553170049593, 0.9999999999197727)

When ratio is 0.7, p = 0.99 > 0.05, we cannot reject H0 and cannot accept Ha.

In [36]:
t5,p5=stats.ttest_rel(df5['decision_tree'], df5['naive_bayes'], alternative = 'less')
t5,p5

(15.607542132755036, 0.9999999600647929)

When ratio is 0.9, p = 0.99 > 0.05, we cannot reject H0 and cannot accept Ha.